In [1]:
import tensorflow as tf
print("GPU available:", tf.config.list_physical_devices('GPU'))
print("Built with CUDA:", tf.test.is_built_with_cuda())

2025-09-05 11:55:57.595176: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-05 11:55:57.634140: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-05 11:55:57.634190: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-05 11:55:57.635603: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-05 11:55:57.642730: I tensorflow/core/platform/cpu_feature_guar

GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Built with CUDA: True


In [1]:
import os
import numpy as np
import tensorflow as tf
from data_loader import DataGenerator
from sklearn.metrics import f1_score, precision_score, recall_score
import datetime
from helpers import f1 as f1_metric
from model import lstm_unet

def set_random_seeds(seed):
    """Set random seeds for reproducibility."""
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

def create_data_generators():
    """Create data generators for training and validation."""
    train_gen = DataGenerator(
        image_dir="../Unet/data_v3_processed/train/images/images",
        mask_dir="../Unet/data_v3_processed/train/masks/masks",
        batch_size=8,
        time_steps=15,
        img_size=(256,256),
        normalize_images=True,
        normalize_masks=False,
        infinite=True,
    )
    
    val_gen = DataGenerator(
        image_dir="../Unet/data_v3_processed/val/images/images",
        mask_dir="../Unet/data_v3_processed/val/masks/masks",
        batch_size=8,
        time_steps=15,
        img_size=(256,256),
        normalize_images=True,
        normalize_masks=False,
        infinite=True
    )

    return train_gen, val_gen

def train_and_evaluate_model(run_number, train_generator, val_generator, 
                           epochs=3, steps_per_epoch=1000, validation_steps=5, 
                           random_seed=None):
    """Train a single model and return evaluation metrics."""
    print(f"\nTraining Run {run_number + 1}/5 (Seed: {random_seed})")
    print("=" * 50)
    
    if random_seed is not None:
        set_random_seeds(random_seed)
    
    model = lstm_unet()
    
    model.compile(optimizer='adam', 
                 loss='binary_crossentropy', 
                 metrics=[f1_metric])
    
    try:
        history = model.fit(train_generator,
                           validation_data=val_generator,
                           epochs=epochs,
                           steps_per_epoch=steps_per_epoch,
                           validation_steps=validation_steps,
                           verbose=1)
        if run_number == 0:
            model_path = os.path.join("models", f"lstm_unet_run4.h5")
        else:
            model_path = os.path.join("models", f"lstm_unet_run5.h5")
        os.makedirs("models", exist_ok=True)
        model.save(model_path)
        
        return model_path
        
    except Exception as e:
        print(f"ERROR in training: {e}")
        return None

# Main execution
random_seeds = [789, 999]

for run in range(2):
    # CREATE FRESH GENERATORS FOR EACH RUN - this fixes the state issue
    train_generator, val_generator = create_data_generators()
    
    model_path = train_and_evaluate_model(
        run, train_generator, val_generator, random_seed=random_seeds[run]
    )
    
    tf.keras.backend.clear_session()

2025-09-05 13:43:31.125251: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-05 13:43:31.162143: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-05 13:43:31.162185: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-05 13:43:31.163617: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-05 13:43:31.171342: I tensorflow/core/platform/cpu_feature_guar


Training Run 1/5 (Seed: 789)


2025-09-05 13:43:33.477585: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2025-09-05 13:43:33.477935: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46672 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:41:00.0, compute capability: 8.9


Epoch 1/3


2025-09-05 13:43:45.454647: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
2025-09-05 13:43:51.523400: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fe4a7f29130 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-09-05 13:43:51.523442: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX 6000 Ada Generation, Compute Capability 8.9
2025-09-05 13:43:51.528037: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1757079831.661720   26693 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1000/1000 [==============================] - 454s 423ms/step - loss: 0.0730 - f1: 0.4626 - val_loss: 0.0162 - val_f1: 0.3266
Epoch 2/3
1000/1000 [==============================] - 420s 419ms/step - loss: 0.0049 - f1: 0.9048 - val_loss: 0.0097 - val_f1: 0.7417
Epoch 3/3
1000/1000 [==============================] - 421s 420ms/step - loss: 0.0027 - f1: 0.9068 - val_loss: 0.0077 - val_f1: 0.7558


/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Training Run 2/5 (Seed: 999)
Epoch 1/3
1000/1000 [==============================] - 440s 422ms/step - loss: 0.0549 - f1: 0.6354 - val_loss: 0.0198 - val_f1: 0.3366
Epoch 2/3
1000/1000 [==============================] - 422s 421ms/step - loss: 0.0032 - f1: 0.9167 - val_loss: 0.0065 - val_f1: 0.7657
Epoch 3/3
1000/1000 [==============================] - 420s 419ms/step - loss: 0.0020 - f1: 0.9240 - val_loss: 0.0457 - val_f1: 0.0000e+00
